In [ ]:
from tokenizers import (
    models,
    normalizers,
    pre_tokenizers,
    trainers,
    Tokenizer,
)
import os 
import math
#训练分词器
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(),normalizers.Lowercase()]
)
tokenizer.pre_tokenizer = pre_tokenizers.WhitespaceSplit()
special_tokens = ["[UNK]","[PAD]","[CLS]","[SEP]","[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size =10000,special_tokens=special_tokens)

data_list = []
for root,paths,names in os.walk("./对联data"):
    for name in names:
        data_list.append(os.path.join(root,name))

tokenizer.train(data_list,trainer=trainer)
tokenizer.save("./model_save/transformer_my_tokenizer.json")



In [8]:
#使用训练好的分词器
tokenizer = Tokenizer.from_file("./model_save/transformer_my_tokenizer.json")
s = "尽其在我，听其在天"
s = " ".join([c for c in s])
tokens = tokenizer.encode(s)
print(tokens.ids,tokens.tokens)

[1787, 462, 1209, 2510, 9026, 840, 462, 1209, 1435] ['尽', '其', '在', '我', '，', '听', '其', '在', '天']


In [17]:
import torch
import json
class ModelConfig:
    def __init__(self,
                device = torch.device("cuda:0"),
                vocab_size = None,
                max_len = None,
                hidden_dim = None,
                n_head = None,
                drop_out = None,
                encoder_layer_count = None,
                decoder_layer_count = None,
                padding_id = None
                ):
                
        self.device = device
        self.vocab_size = vocab_size
        self.max_len = max_len
        self.hidden_dim = hidden_dim
        self.n_head = n_head
        self.drop_out = drop_out
        self.encoder_layer_count = encoder_layer_count
        self.decoder_layer_count = decoder_layer_count
        self.padding_id = padding_id

    
    def save(self,path):
        f = open(path,'w')
        d = {
            "vocab_size":self.vocab_size,
            "max_len":self.max_len,
            "hidden_dim":self.hidden_dim,
            "n_head":self.n_head,
            "drop_out":self.drop_out,
            "encoder_layer_count":self.encoder_layer_count,
            "decoder_layer_count":self.decoder_layer_count,
            "padding_id":self.padding_id,
        }
        d = json.dumps(d)
        f.write(d)
        f.close()

    def load(self,path):
        d = open(path).read()
        d = json.loads(d)
        self.vocab_size = d["vocab_size"]
        self.max_len = d["max_len"]
        self.hidden_dim = d["hidden_dim"]
        self.n_head = d["n_head"]
        self.drop_out = d["drop_out"]
        self.encoder_layer_count = d["encoder_layer_count"]
        self.decoder_layer_count = d["decoder_layer_count"]
        self.padding_id = d["padding_id"]

In [9]:
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
import json
import numpy as np
import random

class Embedding(nn.Module):
    def __init__(self,config):
        super(Embedding,self).__init__()
        #id编码
        self.word_embeddings = nn.Embedding(num_embeddings = config.vocab_size,embedding_dim = config.hidden_dim).to(config.device)
        self.position_embeddings = nn.Embedding(num_embeddings = config.max_len,embedding_dim = config.hidden_dim).to(config.device)
        self.token_type_embeddings = nn.Embedding(num_embeddings = 2,embedding_dim = config.hidden_dim).to(config.device)
        self.device = config.device

    def forward(self,input_ids,token_type_ids=None):
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length,dtype = torch.long,device = self.device)
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids,device = self.device)

        we = self.word_embeddings(input_ids)
        pe = self.position_embeddings(position_ids)
        te = self.token_type_embeddings(token_type_ids)
        return we + pe + te

In [23]:
class MultiHeadAttention(nn.Module):
    def __init__(self,config):
        super(MultiHeadAttention,self).__init__()
        self.n_head = config.n_head
        self.dim = config.hidden_dim
        self.device = config.device

        self.wq = nn.Linear(self.dim,self.dim).to(self.device)
        self.wk = nn.Linear(self.dim,self.dim).to(self.device)
        self.wv = nn.Linear(self.dim,self.dim).to(self.device)

        self.softmax = nn.Softmax(dim=3)
        self.f = nn.Linear(self.dim,self.dim).to(self.device)
        self.norm = nn.LayerNorm(self.dim).to(self.device)
        self.dropout = nn.Dropout(config.drop_out)

    def concat(self,tensor):
        a, b, c, d = tensor.size()
        return tensor.view(a,c,b*d)
    
    def attention(self,q,k,v):
        _, _, _, d = q.size()
        kt = k.transpose(2,3)
        s = (q @ kt)/math.sqrt(d)
        s = self.softmax(s)
        v = s @ v
        return v
    
    def split(self,tensor):
        a,b,c = tensor.size()
        d = c // self.n_head
        return tensor.view(a,self.n_head,b,d)
    
    def forward(self,x,encode_output=None):
        if encode_output is not None:
            q,k,v = self.wq(encode_output),self.wk(x),self.wv(encode_output)
        else:
            q,k,v = self.wq(x),self.wk(x),self.wv(x)
        q,k,v = self.split(q),self.split(k),self.split(v)
        v = self.attention(q,k,v)
        v = self.concat(v)
        vb = v
        v = self.norm(x+v)
        v = self.f(v)
        v = self.norm(vb+v)
        return self.dropout(v)

In [14]:
class Encoder(nn.Module):
    def __init__(self,config):
        super(Encoder,self).__init__()
        self.attention = MultiHeadAttention(config)
        
    def forward(self,x):
        encoder_output = self.attention(x)
        return encoder_output

In [21]:
class Decoder(nn.Module):
    def __init__(self,config):
        super(Decoder,self).__init__()
        self.attention = MultiHeadAttention(config)
        self.encode_decode_attention = MultiHeadAttention(config)

    def forward(self,x,encode_out):
        x = self.attention(x)
        decode_out = self.encode_decode_attention(x,encode_out)
        return decode_out

In [16]:
class Transformer(nn.Module):
    def __init__(self,config):
        super(Transformer,self).__init__()
        self.embedding = Embedding(config)
        self.encoders = nn.ModuleList([Encoder(config) for _ in range(config.encoder_layer_count)])
        self.decoders = nn.ModuleList([Decoder(config) for _ in range(config.decoder_layer_count)])
        self.f = nn.Linear(config.hidden_dim,config.vocab_size).to(config.device)

    def forward(self,src_ids,drc_ids):
        em = self.embedding(src_ids)
        for encoder in self.encoders:
            em = encoder(em)
        dem = self.embedding(drc_ids)
        for decoder in self.decoders:
            dem = decoder(dem,em)
        dem = self.f(dem)
        return dem

In [26]:
class CoupleDataset(Dataset):
    def __init__(self,path):
        self.tokenizer = Tokenizer.from_file("./model_save/transformer_my_tokenizer.json")
        in_data = open(os.path.join(path,"in.txt"),encoding="utf8").readlines()
        out_data = open(os.path.join(path,"out.txt"),encoding="utf8").readlines()

        data = []
        for i in tqdm(range(len(in_data))):
            s,t = in_data[i][:30],out_data[i][:30]
            if len(s)==0 or len(t) == 0:
                continue
            sids = self.tokenizer.encode(s).ids
            tids = self.tokenizer.encode(t).ids
            size = len(tids)
            for j in range(size):
                data.append([sids,[3]+tids[:size-j]])
            self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        s,t = self.data[index]
        if len(s) < 30:
            s = s + [1 for _ in range(30 - len(s))]
        if len(t) < 31:
            t = t + [1 for _ in range(31 - len(t))]
        return np.array(s,dtype=np.int64),np.array(t,dtype=np.int64)

def train():
    config = ModelConfig(device=torch.device("cuda:0"))
    config.load("./model_save/transformer_config.json")
    model = Transformer(config)
    data = CoupleDataset("./对联data/train")
    loader = DataLoader(data,batch_size=20,shuffle=True)
    optimizer = optim.Adam(model.parameters(),lr=1e-4)
    criterion = torch.nn.CrossEntropyLoss(ignore_index=config.padding_id)
    epoch = 20
    step = 0
    for e in range(epoch):
        pbar = tqdm(loader)
        for s,t in pbar:
            s = s.to(config.device)
            t = t.to(config.device)
            predict = model(s,t[:,:-1])
            predict_v = predict.contiguous().view(-1,predict.shape[-1])
            y = t[:,1:].contiguous().view(-1)
            optimizer.zero_grad()
            loss = criterion(predict_v,y)
            loss.backward()
            optimizer.step()
            desc = "[{}/{}][{}][{}]".format(e+1,epoch,step,float(loss))
            pbar.set_description(desc)
            step+=1
            if step% 100 ==0:
                pids = torch.argmax(predict,dim=2)
                y = t[:,1:]
                for i in range(predict.size(0)):
                    ss = data.tokenizer.decode(s.detach().cpu().numpy()[i])
                    ps = data.tokenizer.decode(pids.detach().cpu().numpy()[i])
                    oris = data.tokenizer.decode(y.detach().cpu().numpy()[i])
                    print("\n\n==[{}]==\n==[{}]==\n[{}]==\n\n".format(ss,ps,oris))

In [27]:
train()

[1/20][104][8.448685646057129]:   0%|          | 105/324516 [00:04<3:33:42, 25.30it/s]



==[梦 寄 湖 心 天 作 画]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[情 耽 月 色 水 吟 诗]==




==[风 月 情 犹 在]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[欢 娱 夜]==




==[如 画 江 山 千 古 秀]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[多]==




==[龙 腰 可 会 生 鸡 眼]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[羊 肉 无 需 挂]==




==[鼓 响 三 更 三 响 鼓]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[灯 明]==




==[北 地 雪 飞 ， 漫 空 飞 雪 天 花 坠]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[东 风]==




==[惨 淡 经 营 任 仔 肩 ， 社 中 巨 擘]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[淋 漓 刻 画]==




==[慕 贤 远 拜 陶 公 柳]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[弘 道 恭 随]==




==[半 亩 此 迎 神 ， 越 国 江 山 留 垄 断]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[五 湖 长 浪 迹 ， 吴 宫 花 草 入]==




==[自 力 更 生 ， 创 千 秋 大 业]==
==[， ， ， ， ， ， ， ， ，

[1/20][204][7.952225685119629]:   0%|          | 204/324516 [00:07<3:25:01, 26.36it/s]



==[湾 畔 寻 芳 ， 兰 亭 小 憩 ， 片 时 成 就 花]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[滩 边 拾 韵]==




==[是 清 绝 杜 鹃 ， 带 露 教 人 生 快 意]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[纵 贫 寒 本]==




==[周 览 原 上 观 美 景]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[燕 飞 堂 前]==




==[百 代 誉 诗 魔 ， 震 古 烁 今 ， 几 多 妙 句]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[九 州 名 雅 望 ， 开 来 继]==




==[翠 岫 作 屏 ， 澄 湖 为 镜]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[白]==




==[金 壶 藉 草 溪 亭 晚]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[竹 坞 无 尘 水 槛 清]==




==[登 高 寻 胜 景]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[坐 大]==




==[夜 咏 茶 经 能 解 酒]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[窗]==




==[无 根 永 固 ， 无 翼 长 飞 ， 问 诸 子 百 家]==
==[， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ， ，]==
[居 德 不 孤 ， 居 功 未 傲 ， 看]==




==[问 诗

[1/20][304][7.507124423980713]:   0%|          | 303/324516 [00:11<3:26:07, 26.22it/s] 

...



==[编 钟 播 文 明 ， 威 楚 莺 歌 燕 舞]==
==[人 千 花 人 人 ， 千 千 山 山 ， 风 风 山 ， ， 山 山 ， ， 山 天 ， 花 花 花 花 花 花 花]==
[铜 鼓 催 奋 进 ， 彝 州]==




==[祥 光 紫 气 门 前 聚]==
==[人 花 无 天 地 无 山 山 山 山 山 山 山 山 山 天 花 ， 地 地 山 风 花 花 花 花 花 花 花 花]==
[福]==




==[创 业 兴 家 ， 财 源 茂 似 三 江 水]==
==[地 人 风 明 ， 人 人 千 山 ， 千 月 月 山 月 ， ， ， ， 山 ， 风 花 花 花 风 花 花 花 花]==
[种 田 免 税 ， 政 策 鲜 如 二 月 花]==




==[缅 思 上 古 圣 神 ， 四 时 钻 燧]==
==[心 心 生 山 心 山 ， 春 山 ， 山 山 山 山 ， 天 山 天 山 ， ， 天 花 花 花 风 花 ， 花 花]==
[请 看 太 平 气]==




==[一 场 春 风 一 场 梦 ， 爱 雨 温 柔 ， 且 向]==
==[有 风 中 万 万 风 人 ， 万 心 ， 风 ， 心 人 天 山 山 山 地 花 花 花 花 花 风 花 花 花 花]==
[几 时 归 客 几 时 来 ， 拂 肩 落 寞 ，]==




==[奇 峰 叠 趣 ， 仙 境 勾 来 骚 客 影]==
==[春 水 心 水 ， 水 水 水 山 山 山 月 月 山 千 天 山 山 山 山 ， 天 花 花 花 风 花 花 花 花]==
[秀 水 含 情 ， 渔 船 载]==




==[庭 阶 日 暖 舒 麟 趾]==
==[人 梦 风 山 风 风 人 风 山 山 山 山 山 山 山 ， 山 ， 花 山 山 风 花 花 花 花 花 花 花 花]==
[桂 阁 风]==




==[千 寻 素 练 悬 云 壁]==
==[万 生 梦 生 万 山 万 山 山 山 山 山 山 山 山 风 山 ， 花 天 山 风 天 花 花 花 花 花 花 花]==
[万 斛 银 珠]==




==[助 人 为 乐 心 乐]==
==[万 风 ， 千 江 万 ， ， 山 山 山 山 山 山 ， 风 山 ， 山 山 ， 风 天 花 花 花 花 花 花 花]==


[1/20][2804][6.388358116149902]:   1%|          | 2805/324516 [01:47<3:28:47, 25.68it/s] 



==[扪 斗 巡 天 ， 振 臂 一 呼 ， 时 令 井 然 从]==
==[开 风 人 风 ， 花 情 风 山 ， 山 山 山 山 山 花 花 风 ， 花 ， 风 花 花 花 花 花 花 花 花]==
[开 炊 司 火 ，]==




==[枕 上 常 留 诗 入 梦]==
==[千 心 花 山 山 ， 山 山 山 山 山 山 花 山 山 风 山 山 山 山 ， 风 花 花 花 ， 花 花 花 花]==
[窗 前]==




==[雪 抱 寒 枝 梅 素 面]==
==[风 有 风 千 风 风 风 风 山 山 山 山 月 山 山 风 花 风 ， 花 山 风 花 花 花 花 花 花 花 花]==
[风 牵 碧 柳 鹤]==




==[天 上 缤 纷 ， 风 月 栽 来 一 夜 锦]==
==[人 人 人 人 ， 人 人 风 千 万 春 春 一 春 春 天 花 风 ， 山 ， 风 花 花 花 花 花 花 花 花]==
[人 间 热 闹 ， 烟 花 并 作 万 家 春]==




==[心 无 俗 累 菊 多 种]==
==[水 有 九 天 一 一 一 一 山 花 山 山 花 山 山 ， 花 ， ， ， ， 风 花 花 花 花 花 花 花 花]==
[庐 对 云]==




==[大 漠 孤 烟 悬 落 日]==
==[月 秋 春 一 万 一 来 秋 山 山 山 山 风 山 风 风 花 风 ， 山 花 风 花 花 花 花 花 花 花 花]==
[寒 秋 连 雁 入 流 云]==




==[沧 桑 成 象 荣 滨 海]==
==[年 花 花 花 春 花 年 春 山 山 山 山 花 山 春 花 花 风 ， ， 花 风 花 花 花 花 花 花 花 花]==
[联]==




==[白 云 添 足 趟 河 过]==
==[绿 中 无 山 春 一 中 山 山 山 山 山 花 山 山 天 花 花 ， 花 花 风 花 花 花 花 花 花 花 花]==
[绿 柳 弯 腰]==




==[临 阁 望 上 京 ， 驰 龙 舞 凤 ， 六 合 惠 物]==
==[一 人 风 人 人 ， 人 一 山 山 ， 山 月 ， ， 天 花 风 花 山 花 花 花 花 花 花 花 ， 山 花]==
[逸 政 垂 青 史 ， 拓]==




==[不 死 妙 方 唯 佛 有]=

[1/20][2839][5.968044281005859]:   1%|          | 2840/324516 [01:49<3:26:37, 25.95it/s] 


KeyboardInterrupt: 